In [35]:
import numpy as np
import parmed as pmd
import os
import signac
import mbuild as mb
project = signac.get_project()

for job in project.find_jobs():
    if job.statepoint()['T'] == 298 and job.statepoint()['Seed'] ==1:
        import os
        import glob
        import numpy as np
        import unyt as u
        import mbuild as mb
        from get_sol_il_xml import GetSolv, GetIL, Get_ff_path
        import ilforcefields.ilforcefields as ilff
        from foyer import Forcefield
        cation = GetIL("Li")
        cation.name = "Li"
        anion = GetIL("TF2")
        anion.name = "TF2"
        sol = GetIL("wat")
        sol.name = "wat"
        
        #### molecule number setting for 10m, 
        n_list = [9, 9, 50] #[Li, TF2, wat]
        print('n_list is ', n_list)
        
        packing_box = mb.Box([7,7,7]) ## note: only for eaiser to fill box, not final box size
        

            
        print("start to fill box")
        box_system = mb.fill_box(
                                    compound = [cation, anion, sol],
                                    n_compounds = [
                                                int(n_list[0]),
                                                int(n_list[1]),
                                                int(n_list[2])
                                                ],
                                    box = packing_box,
                                    edge=0.02
                                    )
        cation_cmp = mb.Compound()
        anion_cmp = mb.Compound()
        sol_cmp = mb.Compound()
        
        for child in box_system.children:
            if child.name == 'Li':
                cation_cmp.add(mb.clone(child))
            elif child.name == 'TF2':
                anion_cmp.add(mb.clone(child))
            elif child.name == 'wat':
                sol_cmp.add(mb.clone(child)) 
        print('start to apply force field') 
        opls_li = Get_ff_path('opls_ions')
        opls_li = Forcefield(opls_li)
        cationPM = opls_li.apply(cation_cmp, residues = 'Li')
        clp = ilff.load_LOPES()
        anionPM = clp.apply(anion_cmp, residues = 'TF2')
        spce = Get_ff_path('spce')
        spce = Forcefield(spce)
        solPM = spce.apply(sol_cmp, residues = 'wat')
        structure = cationPM + anionPM + solPM
        
        print('start to scale partial charge to 0.8')
        for atom in structure.atoms:
            if atom.residue.name in ['Li','TF2']:
                atom.charge *= 0.8
                
        structure.combining_rule = 'geometric'
        
        init_pos_struc = mb.load('init_structures/struc_10m_small_{}K_{}.xyz'.format(int(job.statepoint()['T']), job.statepoint()['Seed']))
        init_pos_struc = init_pos_struc.to_parmed()
        pos = init_pos_struc.coordinates
        
        ### copy positions from cp2k to current md structure
        for i, atom in enumerate(structure.atoms):
            atom.xx = pos[i][0]
            atom.xy = pos[i][1]
            atom.xz = pos[i][2]
            
        if job.statepoint()['T'] == 298:   
            box_size = [1.54361, 1.53940,1.54380]
            structure.box[0] = box_size[0] * 10
            structure.box[1] = box_size[1] * 10
            structure.box[2] = box_size[2] * 10
        
        if job.statepoint()['T'] == 373: 
            box_size = [1.57932,1.57932,1.57932]
            structure.box[0] = box_size[0] * 10
            structure.box[1] = box_size[1] * 10
            structure.box[2] = box_size[2] * 10
        
        structure.save(os.path.join(job.workspace(), "system.gro"), combine ="all", overwrite=True)
        structure.save(os.path.join(job.workspace(), "system.top"), combine ="all", overwrite=True)
        
        break
        

n_list is  [9, 9, 50]
start to fill box
start to apply force field


/raid6/homes/linx6/.conda/envs/myenv/lib/python3.7/site-packages/parmed/openmm/topsystem.py:242: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  'all be present in one form or another.', OpenMMWarning)


start to scale partial charge to 0.8


In [29]:
# for atom in init_pos_struc.atoms:
#     print(atom.name, atom.xx, atom.xy, atom.xz)

Li 0.0 0.0 0.0
Li 0.6655 -5.1906 2.4376
Li 0.598 6.850290000000001 2.096
Li 2.9183 -3.8861 -5.27784
Li 5.944060000000001 -4.5419 6.220560000000001
Li 2.4217 -6.88602 -1.16441
Li 7.29615 6.585390000000001 6.83006
Li -4.66015 0.50236 -2.13594
Li -1.076 4.72346 7.270650000000001
O 5.157790000000001 -5.988100000000001 1.1612
F 8.67558 -4.8911 -0.0096
C 7.753770000000001 -5.6757 0.613
S 6.06477 -4.9673 0.5427
O 5.84066 -4.5367 -0.8525500000000001
F 7.81872 -6.87154 -0.0162
N 5.713240000000001 -3.5286 1.5284
O 7.50202 -2.4587 0.2901
F 8.17867 -5.9224 1.8179
S 6.787990000000001 -2.2958 1.5682
O 5.977040000000001 -1.0771 1.7762
F 8.85451 -3.4527 2.8755
F 7.5366 -2.1789000000000005 4.0551
C 8.05016 -2.3623 2.8513000000000006
F 8.92181 -1.3402 2.7142
O -0.1559 3.12592 -4.01577
F -1.7515000000000003 0.6991000000000002 -4.4673
C -1.3356 1.21857 -5.623900000000001
S -0.0624 2.46172 -5.31253
O -0.0001 3.41122 -6.44259
F -0.8173000000000001 0.29477 -6.40586
N 1.6009000000000002 1.85384 -5.42552000000

In [32]:
init_pos_struc.coordinates

array([[ 0.00000e+00,  0.00000e+00,  0.00000e+00],
       [ 6.65500e-01, -5.19060e+00,  2.43760e+00],
       [ 5.98000e-01,  6.85029e+00,  2.09600e+00],
       [ 2.91830e+00, -3.88610e+00, -5.27784e+00],
       [ 5.94406e+00, -4.54190e+00,  6.22056e+00],
       [ 2.42170e+00, -6.88602e+00, -1.16441e+00],
       [ 7.29615e+00,  6.58539e+00,  6.83006e+00],
       [-4.66015e+00,  5.02360e-01, -2.13594e+00],
       [-1.07600e+00,  4.72346e+00,  7.27065e+00],
       [ 5.15779e+00, -5.98810e+00,  1.16120e+00],
       [ 8.67558e+00, -4.89110e+00, -9.60000e-03],
       [ 7.75377e+00, -5.67570e+00,  6.13000e-01],
       [ 6.06477e+00, -4.96730e+00,  5.42700e-01],
       [ 5.84066e+00, -4.53670e+00, -8.52550e-01],
       [ 7.81872e+00, -6.87154e+00, -1.62000e-02],
       [ 5.71324e+00, -3.52860e+00,  1.52840e+00],
       [ 7.50202e+00, -2.45870e+00,  2.90100e-01],
       [ 8.17867e+00, -5.92240e+00,  1.81790e+00],
       [ 6.78799e+00, -2.29580e+00,  1.56820e+00],
       [ 5.97704e+00, -1.07710e

In [33]:
for i, atom in enumerate(structure.atoms):
    print(i, atom.name, atom.xx, atom.xy, atom.xz)

0 Li 51.373623 45.269092 18.605709
1 Li 22.067407000000003 44.442554 15.122919000000001
2 Li 1.0448180000000002 19.858188 60.195609000000005
3 Li 40.513789 52.311877 29.456795000000003
4 Li 43.085865 38.471328 53.78134200000001
5 Li 65.206776 30.440241 38.954256
6 Li 5.434058 42.04455200000001 22.645877
7 Li 24.631211000000004 66.334001 63.424645
8 Li 2.9319160000000006 4.895919 57.01432700000001
9 O 60.68085700000001 32.253042 19.832770000000004
10 F 59.775814 28.517505 19.41353
11 C 60.508915 29.652245 19.246577000000002
12 S 59.711096 31.121781 19.997204000000004
13 O 59.271238000000004 30.723769 21.350204
14 F 61.707437 29.390711 19.81683
15 N 58.25410900000001 31.77992 19.216739
16 O 57.28354 29.587734 19.577566
17 F 60.776194 29.779651000000005 17.979488
18 S 57.08064100000001 30.759572 18.708371
19 O 55.819900000000004 31.522338000000005 18.830572
20 F 58.344658 29.483800000000002 16.685367
21 F 57.000234 31.108203000000003 16.13335
22 C 57.21274700000001 30.150257000000003 17.0

In [14]:
for atom in system.particles():
    print(atom.name, atom.pos)


Li [0. 0. 0.]
Li [ 0.06655 -0.51906  0.24376]
Li [0.0598   0.685029 0.2096  ]
Li [ 0.29183  -0.38861  -0.527784]
Li [ 0.594406 -0.45419   0.622056]
Li [ 0.24217  -0.688602 -0.116441]
Li [0.729615 0.658539 0.683006]
Li [-0.466015  0.050236 -0.213594]
Li [-0.1076    0.472346  0.727065]
O [ 0.515779 -0.59881   0.11612 ]
F [ 0.867558 -0.48911  -0.00096 ]
C [ 0.775377 -0.56757   0.0613  ]
S [ 0.606477 -0.49673   0.05427 ]
O [ 0.584066 -0.45367  -0.085255]
F [ 0.781872 -0.687154 -0.00162 ]
N [ 0.571324 -0.35286   0.15284 ]
O [ 0.750202 -0.24587   0.02901 ]
F [ 0.817867 -0.59224   0.18179 ]
S [ 0.678799 -0.22958   0.15682 ]
O [ 0.597704 -0.10771   0.17762 ]
F [ 0.885451 -0.34527   0.28755 ]
F [ 0.75366 -0.21789  0.40551]
C [ 0.805016 -0.23623   0.28513 ]
F [ 0.892181 -0.13402   0.27142 ]
O [-0.01559   0.312592 -0.401577]
F [-0.17515  0.06991 -0.44673]
C [-0.13356   0.121857 -0.56239 ]
S [-0.00624   0.246172 -0.531253]
O [-1.00000e-05  3.41122e-01 -6.44259e-01]
F [-0.08173   0.029477 -0.640586